# Inventory Analytics Metric Views

This notebook contains the SQL DDL statements to create all 10 inventory analytics metric views in Databricks.

**Catalog/Schema:** `juan_dev.retail`  
**Purpose:** Pre-aggregated KPIs for Inventory Analytics Genie Room  
**Created:** 2025-01-27

## Overview

Metric views provide pre-aggregated KPIs that improve query performance and ensure consistent calculations across the Genie room's 11 question types. Each metric view defines:
- **Dimensions**: Attributes for grouping and filtering (e.g., Location, Product Category, Date)
- **Measures**: Pre-defined aggregations (e.g., Total Quantity, Average Days of Supply, Stockout Count)
- **Source Data**: The underlying tables and transformations

**Note:** Execute each cell in order to create the metric views. Metric views use the `WITH METRICS` syntax and YAML language for defining dimensions and measures.


## 1. Inventory Current Status Metric View

**Purpose:** Current inventory status metrics (optimized for Question Type 1)  
**Source:** `gold_inventory_fact` (filtered to latest date)  
**Performance:** Pre-filters to latest date and pre-joins dimensions for faster queries

**Use Cases:**
- "What is the current inventory level by location?"
- "Show me current stock status by product category"
- "What is the total inventory value across all locations?"


In [0]:

-- Inventory Current Status Metric View
CREATE OR REPLACE VIEW juan_dev.retail.inventory_current_status_mv
WITH METRICS
LANGUAGE YAML
AS $$
  version: 1.1
  comment: "Current inventory status metrics by location and product category. Pre-filtered to latest date for optimal performance. Use this view for questions about current stock levels, stockout status, and inventory value."
  source: |
    SELECT 
      i.*,
      l.location_name,
      l.region,
      p.category_level_1 as product_category
    FROM juan_dev.retail.gold_inventory_fact i
    JOIN juan_dev.retail.gold_location_dim l ON i.location_key = l.location_key
    JOIN juan_dev.retail.gold_product_dim p ON i.product_key = p.product_key
    WHERE i.date_key = (SELECT MAX(date_key) FROM juan_dev.retail.gold_inventory_fact)
      AND l.is_active = TRUE
      AND p.is_active = TRUE
  dimensions:
    - name: Location Name
      expr: location_name
      comment: "Physical location name where inventory is held"
    - name: Region
      expr: region
      comment: "Geographic region for location grouping and regional analysis"
    - name: Product Category
      expr: product_category
      comment: "Product category level 1 for category-based inventory analysis"
    - name: Stockout Status
      expr: CASE WHEN is_stockout THEN 'Out of Stock' ELSE 'In Stock' END
      comment: "Current stockout status - filter for Out of Stock to identify stockout products"
    - name: Overstock Status
      expr: CASE WHEN is_overstock THEN 'Overstocked' ELSE 'Normal' END
      comment: "Overstock status indicator - filter for Overstocked to identify excess inventory"
  measures:
    - name: Total Quantity Available
      expr: SUM(quantity_available)
      comment: "Total units available for sale across all products and locations. Use for current inventory level analysis."
    - name: Total Quantity On Hand
      expr: SUM(quantity_on_hand)
      comment: "Total physical inventory units including reserved, damaged, and available inventory."
    - name: Total Inventory Value Cost
      expr: SUM(inventory_value_cost)
      comment: "Total inventory value at cost basis. Use for financial inventory investment analysis."
    - name: Total Inventory Value Retail
      expr: SUM(inventory_value_retail)
      comment: "Total inventory value at retail price. Use for potential revenue analysis."
    - name: Total Margin Value
      expr: SUM(inventory_value_retail - inventory_value_cost)
      comment: "Total potential margin from current inventory (retail value minus cost)."
    - name: Stockout Product Count
      expr: COUNT(CASE WHEN is_stockout THEN 1 END)
      comment: "Number of products currently out of stock. Use to identify stockout frequency by location or category."
    - name: Overstock Product Count
      expr: COUNT(CASE WHEN is_overstock THEN 1 END)
      comment: "Number of products with excess inventory. Use to identify overstock situations."
    - name: Product Count
      expr: COUNT(*)
      comment: "Total number of product-location combinations. Use for inventory breadth analysis."
    - name: Average Days of Supply
      expr: AVG(days_of_supply)
      comment: "Average days until stockout based on current inventory and demand. Lower values indicate higher stockout risk."
$$


## 2. Inventory Stockout Risk Metric View

**Purpose:** Stockout risk analysis with standardized risk levels (optimized for Question Type 2)  
**Source:** `gold_inventory_fact` (filtered to latest date, products with quantity > 0)  
**Performance:** Pre-calculates risk levels and reorder status for consistent risk assessment

**Use Cases:**
- "Which products are at risk of stockout?"
- "Show me high risk products by location"
- "What products are below reorder point?"


In [0]:

-- Inventory Stockout Risk Metric View
CREATE OR REPLACE VIEW juan_dev.retail.inventory_stockout_risk_mv
WITH METRICS
LANGUAGE YAML
AS $$
  version: 1.1
  comment: "Stockout risk analysis with standardized risk levels based on days of supply and reorder points. Use this view for questions about products at risk, reorder needs, and stockout prevention."
  source: |
    SELECT 
      i.*,
      l.location_name,
      l.region,
      p.category_level_1 as product_category,
      p.product_name
    FROM juan_dev.retail.gold_inventory_fact i
    JOIN juan_dev.retail.gold_location_dim l ON i.location_key = l.location_key
    JOIN juan_dev.retail.gold_product_dim p ON i.product_key = p.product_key
    WHERE i.date_key = (SELECT MAX(date_key) FROM juan_dev.retail.gold_inventory_fact)
      AND i.quantity_available > 0
      AND l.is_active = TRUE
      AND p.is_active = TRUE
  dimensions:
    - name: Location Name
      expr: location_name
      comment: "Physical location name for location-specific risk analysis"
    - name: Region
      expr: region
      comment: "Geographic region for regional risk assessment and comparisons"
    - name: Product Category
      expr: product_category
      comment: "Product category level 1 for category-based risk analysis"
    - name: Risk Level
      expr: |
        CASE 
          WHEN days_of_supply <= 3 THEN 'High Risk'
          WHEN days_of_supply <= 7 THEN 'Medium Risk'
          ELSE 'Low Risk'
        END
      comment: "Standardized risk level: High (<=3 days), Medium (<=7 days), Low (>7 days). Filter for High Risk to identify urgent stockout risks."
    - name: Reorder Status
      expr: |
        CASE 
          WHEN quantity_available < reorder_point THEN 'Below Reorder Point'
          WHEN quantity_available = reorder_point THEN 'At Reorder Point'
          ELSE 'Above Reorder Point'
        END
      comment: "Reorder point status. Filter for Below Reorder Point to identify products needing immediate reorder."
  measures:
    - name: Products at Risk Count
      expr: COUNT(*)
      comment: "Number of products at various risk levels. Use with Risk Level dimension to count high-risk products."
    - name: Average Days of Supply
      expr: AVG(days_of_supply)
      comment: "Average days until stockout. Lower values indicate higher urgency for replenishment."
    - name: Products Below Reorder Point Count
      expr: COUNT(CASE WHEN quantity_available < reorder_point THEN 1 END)
      comment: "Number of products below reorder point threshold. These products need immediate reorder action."
    - name: Total Quantity at Risk
      expr: SUM(CASE WHEN days_of_supply <= 7 THEN quantity_available ELSE 0 END)
      comment: "Total inventory units at medium or high risk of stockout (<=7 days supply)."
    - name: Average Reorder Quantity Needed
      expr: AVG(CASE WHEN quantity_available < reorder_point THEN reorder_quantity ELSE 0 END)
      comment: "Average reorder quantity for products below reorder point. Use for replenishment planning."
    - name: Products with No Replenishment Scheduled
      expr: COUNT(CASE WHEN next_replenishment_date IS NULL AND days_of_supply <= 7 THEN 1 END)
      comment: "Count of at-risk products without scheduled replenishment. These require urgent attention."
$$


## 3. Inventory Overstock Analysis Metric View

**Purpose:** Overstock identification with severity levels (optimized for Question Type 3)  
**Source:** `gold_inventory_fact` (filtered to latest date, overstocked products)  
**Performance:** Pre-filters to overstock conditions and calculates severity levels

**Use Cases:**
- "What products are overstocked?"
- "Show me severe overstock situations by location"
- "Which categories have excess inventory?"


In [0]:

-- Inventory Overstock Analysis Metric View
CREATE OR REPLACE VIEW juan_dev.retail.inventory_overstock_analysis_mv
WITH METRICS
LANGUAGE YAML
AS $$
  version: 1.1
  comment: "Overstock identification with severity levels based on days of supply thresholds. Use this view for questions about excess inventory, overstocked products, and inventory optimization opportunities."
  source: |
    SELECT 
      i.*,
      l.location_name,
      l.region,
      p.category_level_1 as product_category,
      p.product_name
    FROM juan_dev.retail.gold_inventory_fact i
    JOIN juan_dev.retail.gold_location_dim l ON i.location_key = l.location_key
    JOIN juan_dev.retail.gold_product_dim p ON i.product_key = p.product_key
    WHERE i.date_key = (SELECT MAX(date_key) FROM juan_dev.retail.gold_inventory_fact)
      AND (i.is_overstock = TRUE OR i.days_of_supply > 60)
      AND l.is_active = TRUE
      AND p.is_active = TRUE
  dimensions:
    - name: Location Name
      expr: location_name
      comment: "Physical location name for location-specific overstock analysis"
    - name: Region
      expr: region
      comment: "Geographic region for regional overstock comparisons"
    - name: Product Category
      expr: product_category
      comment: "Product category level 1 for category-based overstock analysis"
    - name: Overstock Severity
      expr: |
        CASE 
          WHEN days_of_supply > 90 THEN 'Severe Overstock'
          WHEN days_of_supply > 60 THEN 'Moderate Overstock'
          ELSE 'Normal'
        END
      comment: "Overstock severity level: Severe (>90 days supply), Moderate (>60 days supply). Filter for Severe to identify highest priority overstock situations."
  measures:
    - name: Overstock Product Count
      expr: COUNT(*)
      comment: "Number of overstocked products. Use with Severity dimension to count severe vs moderate overstock."
    - name: Total Overstock Quantity
      expr: SUM(quantity_on_hand)
      comment: "Total units of overstocked inventory. Use for inventory reduction planning."
    - name: Total Overstock Value Cost
      expr: SUM(inventory_value_cost)
      comment: "Total cost value of overstocked inventory. Represents capital tied up in excess inventory."
    - name: Average Days of Supply
      expr: AVG(days_of_supply)
      comment: "Average days of supply for overstocked products. Higher values indicate more severe overstock."
    - name: Products Above 90 Days Supply
      expr: COUNT(CASE WHEN days_of_supply > 90 THEN 1 END)
      comment: "Count of products with severe overstock (>90 days supply). These are highest priority for clearance or redistribution."
$$


## 4. Inventory Value Summary Metric View

**Purpose:** Inventory value metrics by location and category (optimized for Question Type 4)  
**Source:** `gold_inventory_fact` (filtered to latest date)  
**Performance:** Pre-aggregates financial metrics for faster value analysis queries

**Use Cases:**
- "What is the inventory value by location?"
- "Show me inventory value by product category"
- "What is the margin value of current inventory?"


In [0]:

-- Inventory Value Summary Metric View
CREATE OR REPLACE VIEW juan_dev.retail.inventory_value_summary_mv
WITH METRICS
LANGUAGE YAML
AS $$
  version: 1.1
  comment: "Inventory value metrics by location, region, and product category. Use this view for questions about inventory value, financial investment in inventory, and margin analysis."
  source: |
    SELECT 
      i.*,
      l.location_name,
      l.region,
      l.location_type,
      p.category_level_1 as product_category_l1,
      p.category_level_2 as product_category_l2
    FROM juan_dev.retail.gold_inventory_fact i
    JOIN juan_dev.retail.gold_location_dim l ON i.location_key = l.location_key
    JOIN juan_dev.retail.gold_product_dim p ON i.product_key = p.product_key
    WHERE i.date_key = (SELECT MAX(date_key) FROM juan_dev.retail.gold_inventory_fact)
      AND l.is_active = TRUE
      AND p.is_active = TRUE
  dimensions:
    - name: Location Name
      expr: location_name
      comment: "Physical location name for location-specific value analysis"
    - name: Region
      expr: region
      comment: "Geographic region for regional inventory value comparisons"
    - name: Location Type
      expr: location_type
      comment: "Location type (Store, Warehouse, Distribution Center) for type-based value analysis"
    - name: Product Category Level 1
      expr: product_category_l1
      comment: "Top-level product category for high-level category value analysis"
    - name: Product Category Level 2
      expr: product_category_l2
      comment: "Second-level product category for detailed category value analysis"
  measures:
    - name: Total Inventory Value Cost
      expr: SUM(inventory_value_cost)
      comment: "Total inventory value at cost. Represents capital invested in inventory."
    - name: Total Inventory Value Retail
      expr: SUM(inventory_value_retail)
      comment: "Total inventory value at retail price. Represents potential revenue from current inventory."
    - name: Total Margin Value
      expr: SUM(inventory_value_retail - inventory_value_cost)
      comment: "Total potential margin from inventory (retail minus cost). Use for profitability analysis."
    - name: Product Count
      expr: COUNT(DISTINCT product_key)
      comment: "Number of distinct products. Use to analyze inventory breadth by location or category."
    - name: Average Inventory Value per Product
      expr: AVG(inventory_value_cost)
      comment: "Average inventory value per product at cost. Use to identify high-value products."
    - name: Average Margin per Product
      expr: AVG(inventory_value_retail - inventory_value_cost)
      comment: "Average margin per product. Use to identify high-margin product categories."
$$


## 5. Inventory Reorder Management Metric View

**Purpose:** Reorder needs and replenishment status (optimized for Question Type 5)  
**Source:** `gold_inventory_fact` (filtered to latest date, products at or below reorder point)  
**Performance:** Pre-filters to products at or below reorder point and calculates replenishment urgency

**Use Cases:**
- "Which products need reordering?"
- "Show me products with no replenishment schedule"
- "What is the reorder status by location?"


In [0]:

-- Inventory Reorder Management Metric View
CREATE OR REPLACE VIEW juan_dev.retail.inventory_reorder_management_mv
WITH METRICS
LANGUAGE YAML
AS $$
  version: 1.1
  comment: "Reorder management with replenishment status and urgency levels. Use this view for questions about products needing reorder, replenishment schedules, and reorder urgency."
  source: |
    SELECT 
      i.*,
      l.location_name,
      l.region,
      p.category_level_1 as product_category,
      p.product_name,
      DATEDIFF(i.next_replenishment_date, CURRENT_DATE) as days_until_replenishment
    FROM juan_dev.retail.gold_inventory_fact i
    JOIN juan_dev.retail.gold_location_dim l ON i.location_key = l.location_key
    JOIN juan_dev.retail.gold_product_dim p ON i.product_key = p.product_key
    WHERE i.date_key = (SELECT MAX(date_key) FROM juan_dev.retail.gold_inventory_fact)
      AND i.quantity_available <= i.reorder_point
      AND l.is_active = TRUE
      AND p.is_active = TRUE
  dimensions:
    - name: Location Name
      expr: location_name
      comment: "Physical location name for location-specific reorder analysis"
    - name: Region
      expr: region
      comment: "Geographic region for regional reorder planning"
    - name: Product Category
      expr: product_category
      comment: "Product category level 1 for category-based reorder analysis"
    - name: Reorder Urgency
      expr: |
        CASE 
          WHEN next_replenishment_date IS NULL THEN 'Critical - No Schedule'
          WHEN DATEDIFF(next_replenishment_date, CURRENT_DATE) > 7 THEN 'Delayed - Schedule >7 Days'
          ELSE 'Scheduled - Within 7 Days'
        END
      comment: "Reorder urgency level: Critical (no schedule), Delayed (>7 days away), Scheduled (within 7 days). Filter for Critical to identify urgent reorder needs."
  measures:
    - name: Products Needing Reorder Count
      expr: COUNT(*)
      comment: "Total number of products at or below reorder point. Use with Urgency dimension to prioritize reorders."
    - name: Total Reorder Quantity Needed
      expr: SUM(reorder_quantity)
      comment: "Total units to reorder across all products. Use for purchase order planning."
    - name: Average Days Until Replenishment
      expr: AVG(CASE WHEN next_replenishment_date IS NOT NULL THEN DATEDIFF(next_replenishment_date, CURRENT_DATE) END)
      comment: "Average days until scheduled replenishment. Negative values indicate overdue replenishments."
    - name: Products with No Schedule Count
      expr: COUNT(CASE WHEN next_replenishment_date IS NULL THEN 1 END)
      comment: "Count of products needing reorder without scheduled replenishment. These require immediate attention."
    - name: Products with Delayed Schedule Count
      expr: COUNT(CASE WHEN DATEDIFF(next_replenishment_date, CURRENT_DATE) > 7 THEN 1 END)
      comment: "Count of products with replenishment scheduled >7 days away. May need expedited ordering."
$$


## 6. Inventory Movement Summary Metric View

**Purpose:** Inventory movement transactions by type (optimized for Question Type 6)  
**Source:** `gold_inventory_movement_fact`  
**Performance:** Pre-aggregates movement transactions by type, date, and location

**Use Cases:**
- "What types of inventory movements occurred last month?"
- "Show me receipt activity by location"
- "What is the transfer volume between locations?"


In [0]:

-- Inventory Movement Summary Metric View
CREATE OR REPLACE VIEW juan_dev.retail.inventory_movement_summary_mv
WITH METRICS
LANGUAGE YAML
AS $$
  version: 1.1
  comment: "Inventory movement summary by type, date, and location. Use this view for questions about inventory movements, receipts, transfers, returns, and adjustments."
  source: |
    SELECT 
      im.*,
      d.calendar_date,
      d.month_name,
      d.year,
      l.location_name,
      l.region
    FROM juan_dev.retail.gold_inventory_movement_fact im
    JOIN juan_dev.retail.gold_date_dim d ON im.date_key = d.date_key
    LEFT JOIN juan_dev.retail.gold_location_dim l ON im.to_location_key = l.location_key
  dimensions:
    - name: Movement Date
      expr: calendar_date
      comment: "Date of inventory movement transaction. Use for time-based movement analysis."
    - name: Month Name
      expr: month_name
      comment: "Month of movement for monthly movement trend analysis"
    - name: Year
      expr: year
      comment: "Year of movement for year-over-year movement comparisons"
    - name: Movement Type
      expr: movement_type
      comment: "Type of movement: SALE, RETURN, RECEIPT, TRANSFER, ADJUSTMENT. Filter to analyze specific movement types."
    - name: Location Name
      expr: location_name
      comment: "Destination location for receipts and transfers. Use for location-specific movement analysis."
    - name: Movement Reason
      expr: movement_reason
      comment: "Reason code for movement (e.g., CUSTOMER_ORDER, STOCK_ADJUSTMENT, DAMAGED). Use for root cause analysis."
  measures:
    - name: Movement Count
      expr: COUNT(*)
      comment: "Total number of movement transactions. Use with Movement Type to count receipts, transfers, etc."
    - name: Total Quantity Moved
      expr: SUM(ABS(quantity))
      comment: "Total units moved (absolute value). Use for movement volume analysis."
    - name: Total Movement Cost
      expr: SUM(total_cost)
      comment: "Total cost of inventory movements. Use for movement financial impact analysis."
    - name: Average Movement Quantity
      expr: AVG(ABS(quantity))
      comment: "Average units per movement transaction. Use to analyze typical movement sizes."
    - name: Average Movement Cost per Unit
      expr: AVG(unit_cost)
      comment: "Average cost per unit moved. Use for unit cost trend analysis."
$$


## 7. Inventory Stockout Impact Metric View

**Purpose:** Stockout events with lost sales impact (optimized for Question Type 7)  
**Source:** `gold_stockout_events`  
**Performance:** Pre-aggregates stockout events with lost sales metrics

**Use Cases:**
- "What is the revenue impact of stockouts?"
- "Show me lost sales by location"
- "Which locations have the highest stockout rates?"


In [0]:

-- Inventory Stockout Impact Metric View
CREATE OR REPLACE VIEW juan_dev.retail.inventory_stockout_impact_mv
WITH METRICS
LANGUAGE YAML
AS $$
  version: 1.1
  comment: "Stockout events with lost sales impact by location, product category, and time period. Use this view for questions about stockout frequency, lost revenue, and stockout duration analysis."
  source: |
    SELECT 
      se.*,
      d.calendar_date as stockout_start_date,
      DATE_TRUNC('MONTH', d.calendar_date) as stockout_month,
      l.location_name,
      l.region,
      p.category_level_1 as product_category,
      p.product_name
    FROM juan_dev.retail.gold_stockout_events se
    JOIN juan_dev.retail.gold_date_dim d ON se.stockout_start_date_key = d.date_key
    JOIN juan_dev.retail.gold_location_dim l ON se.location_key = l.location_key
    JOIN juan_dev.retail.gold_product_dim p ON se.product_key = p.product_key
    WHERE l.is_active = TRUE
      AND p.is_active = TRUE
  dimensions:
    - name: Stockout Start Date
      expr: stockout_start_date
      comment: "Date when stockout event began. Use for time-based stockout trend analysis."
    - name: Stockout Month
      expr: stockout_month
      comment: "Month of stockout event for monthly stockout pattern analysis"
    - name: Location Name
      expr: location_name
      comment: "Physical location where stockout occurred. Use for location-specific stockout analysis."
    - name: Region
      expr: region
      comment: "Geographic region for regional stockout comparisons"
    - name: Product Category
      expr: product_category
      comment: "Product category level 1 for category-based stockout analysis"
    - name: Peak Season Flag
      expr: CASE WHEN peak_season_flag THEN 'Peak Season' ELSE 'Regular Season' END
      comment: "Indicates if stockout occurred during peak shopping season. Filter to analyze peak season impact."
    - name: Stockout Duration Category
      expr: |
        CASE 
          WHEN stockout_duration_days <= 7 THEN 'Short Duration (<=7 days)'
          ELSE 'Extended Duration (>7 days)'
        END
      comment: "Stockout duration category. Filter for Extended Duration to identify chronic stockout issues."
  measures:
    - name: Stockout Event Count
      expr: COUNT(*)
      comment: "Total number of stockout events. Use with dimensions to analyze stockout frequency by location, category, or time."
    - name: Total Lost Sales Revenue
      expr: SUM(lost_sales_revenue)
      comment: "Total revenue lost due to stockouts. Primary metric for financial impact of stockouts."
    - name: Total Lost Sales Quantity
      expr: SUM(lost_sales_quantity)
      comment: "Total units requested but unavailable during stockouts. Use for demand impact analysis."
    - name: Average Stockout Duration Days
      expr: AVG(stockout_duration_days)
      comment: "Average number of days products were out of stock. Higher values indicate longer stockout periods."
    - name: Total Lost Sales Attempts
      expr: SUM(lost_sales_attempts)
      comment: "Total number of customer purchase attempts during stockouts. Represents customer frustration events."
    - name: Peak Season Lost Revenue
      expr: SUM(CASE WHEN peak_season_flag THEN lost_sales_revenue ELSE 0 END)
      comment: "Revenue lost during peak shopping seasons. Use to quantify peak season stockout impact."
$$


## 8. Inventory Location Comparison Metric View

**Purpose:** Cross-location inventory metrics comparison (optimized for Question Type 8)  
**Source:** `gold_inventory_fact` (filtered to latest date_key)  
**Performance:** Pre-aggregates metrics by location for faster cross-location comparisons

**Use Cases:**
- "Compare inventory metrics across locations"
- "Which locations have the best inventory health?"
- "Show me regional inventory distribution"


In [0]:

-- Inventory Location Comparison Metric View
CREATE OR REPLACE VIEW juan_dev.retail.inventory_location_comparison_mv
WITH METRICS
LANGUAGE YAML
AS $$
  version: 1.1
  comment: "Cross-location inventory metrics for comparing inventory health, stockouts, and overstock across locations and regions. Use this view for questions about location comparisons and regional inventory analysis."
  source: |
    SELECT 
      i.*,
      l.region,
      l.location_name,
      l.location_type
    FROM juan_dev.retail.gold_inventory_fact i
    JOIN juan_dev.retail.gold_location_dim l ON i.location_key = l.location_key
    WHERE i.date_key = (SELECT MAX(date_key) FROM juan_dev.retail.gold_inventory_fact)
      AND l.is_active = TRUE
  dimensions:
    - name: Region
      expr: region
      comment: "Geographic region for regional inventory comparisons. Use to compare inventory metrics across regions."
    - name: Location Name
      expr: location_name
      comment: "Physical location name for location-specific analysis. Use to identify best and worst performing locations."
    - name: Location Type
      expr: location_type
      comment: "Location type (Store, Warehouse, Distribution Center) for type-based comparisons"
  measures:
    - name: Product Count
      expr: COUNT(DISTINCT product_key)
      comment: "Number of distinct products at each location. Use to analyze inventory breadth by location."
    - name: Total Quantity Available
      expr: SUM(quantity_available)
      comment: "Total units available for sale at each location. Use for inventory level comparisons."
    - name: Total Inventory Value Cost
      expr: SUM(inventory_value_cost)
      comment: "Total inventory investment at cost by location. Use for capital allocation analysis."
    - name: Stockout Product Count
      expr: COUNT(CASE WHEN is_stockout THEN 1 END)
      comment: "Number of products out of stock at each location. Use to identify locations with stockout issues."
    - name: Overstock Product Count
      expr: COUNT(CASE WHEN is_overstock THEN 1 END)
      comment: "Number of overstocked products at each location. Use to identify locations with excess inventory."
    - name: Stockout Rate
      expr: (COUNT(CASE WHEN is_stockout THEN 1 END) * 100.0) / NULLIF(COUNT(*), 0)
      comment: "Percentage of products out of stock at location. Use to compare stockout performance across locations."
    - name: Overstock Rate
      expr: (COUNT(CASE WHEN is_overstock THEN 1 END) * 100.0) / NULLIF(COUNT(*), 0)
      comment: "Percentage of products overstocked at location. Use to identify locations needing inventory redistribution."
    - name: Average Days of Supply
      expr: AVG(days_of_supply)
      comment: "Average days of supply across products at location. Use to compare inventory efficiency."
$$


## 9. Inventory Trends Daily Metric View

**Purpose:** Daily inventory trends over time (optimized for Question Type 9)  
**Source:** `gold_inventory_fact` with date dimension  
**Performance:** Pre-aggregates daily metrics for time-series analysis

**Use Cases:**
- "How have inventory levels changed over time?"
- "Show me monthly inventory trends"
- "What are the seasonal inventory patterns?"


In [0]:

-- Inventory Trends Daily Metric View
CREATE OR REPLACE VIEW juan_dev.retail.inventory_trends_daily_mv
WITH METRICS
LANGUAGE YAML
AS $$
  version: 1.1
  comment: "Daily inventory trends with temporal attributes for time-series analysis. Use this view for questions about inventory trends over time, seasonal patterns, and historical inventory analysis."
  source: |
    SELECT 
      i.*,
      d.calendar_date,
      d.month_name,
      d.year,
      d.quarter,
      d.season,
      d.is_peak_season
    FROM juan_dev.retail.gold_inventory_fact i
    JOIN juan_dev.retail.gold_date_dim d ON i.date_key = d.date_key
  dimensions:
    - name: Calendar Date
      expr: calendar_date
      comment: "Date of inventory snapshot. Use for daily trend analysis and time-based filtering."
    - name: Month Name
      expr: month_name
      comment: "Month name for monthly inventory trend analysis and month-over-month comparisons"
    - name: Year
      expr: year
      comment: "Year for year-over-year inventory comparisons"
    - name: Quarter
      expr: quarter
      comment: "Quarter (1-4) for quarterly inventory trend analysis"
    - name: Season
      expr: season
      comment: "Season (Spring, Summer, Fall, Winter) for seasonal inventory pattern analysis"
    - name: Peak Season Indicator
      expr: CASE WHEN is_peak_season THEN 'Peak Season' ELSE 'Regular Season' END
      comment: "Indicates peak shopping season periods. Use to compare peak vs regular season inventory levels."
  measures:
    - name: Product Count
      expr: COUNT(DISTINCT product_key)
      comment: "Number of distinct products on each date. Use to track product assortment changes over time."
    - name: Total Quantity Available
      expr: SUM(quantity_available)
      comment: "Total units available on each date. Use for inventory level trend analysis."
    - name: Total Inventory Value Cost
      expr: SUM(inventory_value_cost)
      comment: "Total inventory value at cost on each date. Use for inventory investment trend analysis."
    - name: Stockout Product Count
      expr: COUNT(CASE WHEN is_stockout THEN 1 END)
      comment: "Number of products out of stock on each date. Use for stockout trend analysis."
    - name: Overstock Product Count
      expr: COUNT(CASE WHEN is_overstock THEN 1 END)
      comment: "Number of overstocked products on each date. Use for overstock trend analysis."
    - name: Average Days of Supply
      expr: AVG(days_of_supply)
      comment: "Average days of supply on each date. Use to track inventory efficiency trends over time."
$$


## 10. Inventory Health Score Metric View

**Purpose:** Overall inventory health metrics with health status (optimized for Question Type 10)  
**Source:** `gold_inventory_fact` (filtered to latest date_key)  
**Performance:** Pre-calculates health metrics and composite health score

**Note:** This view computes health status in the source query to avoid aggregate functions in dimensions, which is not allowed in metric views.

**Use Cases:**
- "What is the overall inventory health score?"
- "Show me inventory health by category"
- "Which locations have critical inventory health?"


In [0]:

-- Inventory Health Score Metric View
CREATE OR REPLACE VIEW juan_dev.retail.inventory_health_score_mv
WITH METRICS
LANGUAGE YAML
AS $$
  version: 1.1
  comment: "Inventory health metrics with composite health score by location and product category. Use this view for questions about overall inventory health, inventory KPIs, and health status assessment."
  source: |
    SELECT 
      i.*,
      l.location_name,
      l.region,
      p.category_level_1 as product_category,
      CASE 
        WHEN i.is_stockout THEN 'Critical'
        WHEN i.days_of_supply <= 3 THEN 'At Risk'
        WHEN i.days_of_supply > 90 THEN 'Excess Inventory'
        WHEN i.is_overstock THEN 'Excess Inventory'
        ELSE 'Healthy'
      END as health_status_raw
    FROM juan_dev.retail.gold_inventory_fact i
    JOIN juan_dev.retail.gold_location_dim l ON i.location_key = l.location_key
    JOIN juan_dev.retail.gold_product_dim p ON i.product_key = p.product_key
    WHERE i.date_key = (SELECT MAX(date_key) FROM juan_dev.retail.gold_inventory_fact)
      AND l.is_active = TRUE
      AND p.is_active = TRUE
  dimensions:
    - name: Location Name
      expr: location_name
      comment: "Physical location name for location-specific health assessment"
    - name: Region
      expr: region
      comment: "Geographic region for regional health comparisons"
    - name: Product Category
      expr: product_category
      comment: "Product category level 1 for category-based health analysis"
    - name: Health Status
      expr: health_status_raw
      comment: "Overall health status: Critical (stockout), At Risk (<=3 days supply), Excess Inventory (>90 days or overstock), Healthy. Filter to identify locations or categories needing attention."
  measures:
    - name: Total Products
      expr: COUNT(*)
      comment: "Total number of product-location combinations. Base metric for health rate calculations."
    - name: Stockout Product Count
      expr: COUNT(CASE WHEN is_stockout THEN 1 END)
      comment: "Number of products currently out of stock. Use with Total Products for stockout rate."
    - name: Overstock Product Count
      expr: COUNT(CASE WHEN is_overstock THEN 1 END)
      comment: "Number of products with excess inventory. Use with Total Products for overstock rate."
    - name: High Risk Product Count
      expr: COUNT(CASE WHEN days_of_supply <= 3 THEN 1 END)
      comment: "Number of products with <=3 days supply. Indicates imminent stockout risk."
    - name: Excess Inventory Product Count
      expr: COUNT(CASE WHEN days_of_supply > 90 THEN 1 END)
      comment: "Number of products with >90 days supply. Indicates severe overstock."
    - name: Average Days of Supply
      expr: AVG(days_of_supply)
      comment: "Average days of supply. Optimal range is typically 14-45 days depending on product category."
    - name: Stockout Rate
      expr: (COUNT(CASE WHEN is_stockout THEN 1 END) * 100.0) / NULLIF(COUNT(*), 0)
      comment: "Percentage of products out of stock. Target is <5% for healthy inventory."
    - name: Overstock Rate
      expr: (COUNT(CASE WHEN is_overstock THEN 1 END) * 100.0) / NULLIF(COUNT(*), 0)
      comment: "Percentage of products overstocked. Target is <10% for efficient inventory."
    - name: Health Score
      expr: |
        GREATEST(0, LEAST(100, 
          100 
          - (COUNT(CASE WHEN is_stockout THEN 1 END) * 10.0 / NULLIF(COUNT(*), 0))
          - (COUNT(CASE WHEN is_overstock THEN 1 END) * 5.0 / NULLIF(COUNT(*), 0))
          - (COUNT(CASE WHEN days_of_supply <= 3 THEN 1 END) * 5.0 / NULLIF(COUNT(*), 0))
        ))
      comment: "Composite health score (0-100). Score = 100 - (stockout_rate * 10) - (overstock_rate * 5) - (high_risk_rate * 5). Higher scores indicate healthier inventory. Target is >80 for good inventory health."
$$



## Metric Views Inventory

### 1. `inventory_current_status_mv`

**File:** `inventory_current_status_mv.sql`  
**Purpose:** Current inventory status metrics (optimized for Question Type 1)  
**Source:** `gold_inventory_fact` (filtered to latest date)

**Use Cases:**
- "What is the current inventory level by location?"
- "Show me current stock status by product category"
- "What is the total inventory value across all locations?"

**Dimensions:**
- Location Name
- Region
- Product Category
- Stockout Status (In Stock / Out of Stock)
- Overstock Status (Normal / Overstocked)

**Measures:**
- Total Quantity Available
- Total Quantity On Hand
- Total Inventory Value Cost
- Total Inventory Value Retail
- Total Margin Value
- Stockout Product Count
- Overstock Product Count
- Product Count
- Average Days of Supply

---

### 2. `inventory_stockout_risk_mv`

**File:** `inventory_stockout_risk_mv.sql`  
**Purpose:** Stockout risk analysis with standardized risk levels (optimized for Question Type 2)  
**Source:** `gold_inventory_fact` (filtered to latest date, products with quantity > 0)

**Use Cases:**
- "Which products are at risk of stockout?"
- "Show me high risk products by location"
- "What products are below reorder point?"

**Dimensions:**
- Location Name
- Region
- Product Category
- Risk Level (High <=3 days, Medium <=7 days, Low >7 days)
- Reorder Status (Below / At / Above Reorder Point)

**Measures:**
- Products at Risk Count
- Average Days of Supply
- Products Below Reorder Point Count
- Total Quantity at Risk
- Average Reorder Quantity Needed
- Products with No Replenishment Scheduled

---

### 3. `inventory_overstock_analysis_mv`

**File:** `inventory_overstock_analysis_mv.sql`  
**Purpose:** Overstock identification with severity levels (optimized for Question Type 3)  
**Source:** `gold_inventory_fact` (filtered to latest date, overstocked products)

**Use Cases:**
- "What products are overstocked?"
- "Show me severe overstock situations by location"
- "Which categories have excess inventory?"

**Dimensions:**
- Location Name
- Region
- Product Category
- Overstock Severity (Severe >90 days, Moderate >60 days)

**Measures:**
- Overstock Product Count
- Total Overstock Quantity
- Total Overstock Value Cost
- Average Days of Supply
- Products Above 90 Days Supply

---

### 4. `inventory_value_summary_mv`

**File:** `inventory_value_summary_mv.sql`  
**Purpose:** Inventory value metrics by location and category (optimized for Question Type 4)  
**Source:** `gold_inventory_fact` (filtered to latest date)

**Use Cases:**
- "What is the inventory value by location?"
- "Show me inventory value by product category"
- "What is the margin value of current inventory?"

**Dimensions:**
- Location Name
- Region
- Location Type
- Product Category Level 1
- Product Category Level 2

**Measures:**
- Total Inventory Value Cost
- Total Inventory Value Retail
- Total Margin Value
- Product Count
- Average Inventory Value per Product
- Average Margin per Product

---

### 5. `inventory_reorder_management_mv`

**File:** `inventory_reorder_management_mv.sql`  
**Purpose:** Reorder needs and replenishment status (optimized for Question Type 5)  
**Source:** `gold_inventory_fact` (filtered to latest date, products at or below reorder point)

**Use Cases:**
- "Which products need reordering?"
- "Show me products with no replenishment schedule"
- "What is the reorder status by location?"

**Dimensions:**
- Location Name
- Region
- Product Category
- Reorder Urgency (Critical: no schedule, Delayed: >7 days, Scheduled: within 7 days)

**Measures:**
- Products Needing Reorder Count
- Total Reorder Quantity Needed
- Average Days Until Replenishment
- Products with No Schedule Count
- Products with Delayed Schedule Count

---

### 6. `inventory_movement_summary_mv`

**File:** `inventory_movement_summary_mv.sql`  
**Purpose:** Inventory movement transactions by type (optimized for Question Type 6)  
**Source:** `gold_inventory_movement_fact`

**Use Cases:**
- "What types of inventory movements occurred last month?"
- "Show me receipt activity by location"
- "What is the transfer volume between locations?"

**Dimensions:**
- Movement Date
- Month Name
- Year
- Movement Type (SALE, RETURN, RECEIPT, TRANSFER, ADJUSTMENT)
- Location Name
- Movement Reason

**Measures:**
- Movement Count
- Total Quantity Moved
- Total Movement Cost
- Average Movement Quantity
- Average Movement Cost per Unit

---

### 7. `inventory_stockout_impact_mv`

**File:** `inventory_stockout_impact_mv.sql`  
**Purpose:** Stockout events with lost sales impact (optimized for Question Type 7)  
**Source:** `gold_stockout_events`

**Use Cases:**
- "What is the revenue impact of stockouts?"
- "Show me lost sales by location"
- "Which locations have the highest stockout rates?"

**Dimensions:**
- Stockout Start Date
- Stockout Month
- Location Name
- Region
- Product Category
- Peak Season Flag
- Stockout Duration Category (Short <=7 days, Extended >7 days)

**Measures:**
- Stockout Event Count
- Total Lost Sales Revenue
- Total Lost Sales Quantity
- Average Stockout Duration Days
- Total Lost Sales Attempts
- Peak Season Lost Revenue

---

### 8. `inventory_location_comparison_mv`

**File:** `inventory_location_comparison_mv.sql`  
**Purpose:** Cross-location inventory metrics comparison (optimized for Question Type 8)  
**Source:** `gold_inventory_fact` (filtered to latest date)

**Use Cases:**
- "Compare inventory metrics across locations"
- "Which locations have the best inventory health?"
- "Show me regional inventory distribution"

**Dimensions:**
- Region
- Location Name
- Location Type

**Measures:**
- Product Count
- Total Quantity Available
- Total Inventory Value Cost
- Stockout Product Count
- Overstock Product Count
- Stockout Rate (%)
- Overstock Rate (%)
- Average Days of Supply

---

### 9. `inventory_trends_daily_mv`

**File:** `inventory_trends_daily_mv.sql`  
**Purpose:** Daily inventory trends over time (optimized for Question Type 9)  
**Source:** `gold_inventory_fact` with date dimension

**Use Cases:**
- "How have inventory levels changed over time?"
- "Show me monthly inventory trends"
- "What are the seasonal inventory patterns?"

**Dimensions:**
- Calendar Date
- Month Name
- Year
- Quarter
- Season
- Peak Season Indicator

**Measures:**
- Product Count
- Total Quantity Available
- Total Inventory Value Cost
- Stockout Product Count
- Overstock Product Count
- Average Days of Supply

---

### 10. `inventory_health_score_mv`

**File:** `inventory_health_score_mv.sql`  
**Purpose:** Overall inventory health metrics with health status (optimized for Question Type 10)  
**Source:** `gold_inventory_fact` (filtered to latest date)

**Use Cases:**
- "What is the overall inventory health score?"
- "Show me inventory health by category"
- "Which locations have critical inventory health?"

**Dimensions:**
- Location Name
- Region
- Product Category
- Health Status (Critical, Poor, At Risk, Excess Inventory, Healthy)

**Measures:**
- Total Products
- Stockout Product Count
- Overstock Product Count
- High Risk Product Count (<=3 days supply)
- Excess Inventory Product Count (>90 days supply)
- Average Days of Supply
- Stockout Rate (%)
- Overstock Rate (%)
- Health Score (0-100 composite score)
